In [1]:
import datetime

import pandas as pd
pd.options.display.max_rows=6

import time
from io import StringIO

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

KEY_FILE_LOCATION = '/home/jubauser1/jzou/key_files/dcm_api_jubapluscc@gmail.com.json'
SCOPES = ['https://www.googleapis.com/auth/dfareporting']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, scopes=SCOPES)

# Build the service object.
service = build('dfareporting', 'v2.8', credentials=credentials)

profile_id = '4067189'

In [6]:
report_ids = {
    #'operating_system': '95966639', 
    #'browser': '97262781',
    #'hour': '97262254',
    #'date': '97263009',
    #'site': '97262746',
    #'placement': '97262840',
    #'creative': '97262844',
    #'creative_size': '97263030',
    #'ad': '97262761',
    'state_city': '97262853',
    'dma': '97263173',
    #'cpm': '101006950',
    #'clickThroughUrl': '97264726',
    'dbmLineItem': '97349012',
    'dbmSite': '97466324',
    #'cpm_callaway': '102491903'
}
end_date = '2017-09-24'

In [7]:
# run reports
file_ids = dict()
for report_name in report_ids.keys():
    report_id = report_ids[report_name]
    
    # get report info
    request_get_report = service.reports().get(profileId=profile_id, reportId=report_id)
    report = request_get_report.execute()
    
    # update end_date
    report['criteria']['dateRange']['startDate'] = '2017-06-02'
    report['criteria']['dateRange']['endDate'] = end_date
    request_update_report = service.reports().update(profileId=profile_id, reportId=report_id, body=report)
    response = request_update_report.execute()
    
    # run report
    request_run_report = service.reports().run(profileId=profile_id, reportId=report_id)
    result = request_run_report.execute()
    print ('Report %s with ID %s, file with ID %s is currently in status "%s".'
        % (report_name, report_id, result['id'], result['status']))
    file_ids[report_name] = result['id']

Report state_city with ID 97262853, file with ID 614834584 is currently in status "PROCESSING".
Report dma with ID 97263173, file with ID 614834585 is currently in status "PROCESSING".
Report dbmLineItem with ID 97349012, file with ID 614838238 is currently in status "PROCESSING".
Report dbmSite with ID 97466324, file with ID 614837850 is currently in status "PROCESSING".


In [7]:
report_ids = {'state_city': 97262853, 'dma': 97263173}
file_ids = {'state_city': 606064173, 'dma': 606068428}

In [9]:
# check if REPORT_AVAILABLE
for report_name in report_ids.keys():
    report_id = report_ids[report_name]
    file_id = file_ids[report_name]
    request_get_file = service.reports().files().get(
        profileId=profile_id, reportId=report_id, fileId=file_id)
    response = request_get_file.execute()
    print('Report: %s, file ID: %s, has status %s.'
                % (report_name, response['id'], response['status']))

Report: state_city, file ID: 614834584, has status REPORT_AVAILABLE.
Report: dma, file ID: 614834585, has status REPORT_AVAILABLE.
Report: dbmLineItem, file ID: 614838238, has status REPORT_AVAILABLE.
Report: dbmSite, file ID: 614837850, has status REPORT_AVAILABLE.


In [16]:
reports_df = dict() 
for report_name in file_ids.keys():
    file_id = file_ids[report_name]
    report_id = report_ids[report_name]
    
    # check if REPORT_AVAILABLE
    request_get_file = service.reports().files().get(
        profileId=profile_id, reportId=report_id, fileId=file_id)
    response = request_get_file.execute()
    while response['status'] == 'PROCESSING':
        time.sleep(10)
        response = request_get_file.execute()
        print ('Report file with ID %s has status %s.'
            % (response['id'],
               response['status']))
    
    # once REPORT_AVAILABLE, download file
    request_get_media = service.files().get_media(reportId=report_id, fileId=file_id)
    file_str = request_get_media.execute()
    header_rows = 10
    #header_rows = 9 if report_name in no_country else 10
    reports_df[report_name] = pd.read_csv(StringIO(file_str.decode('utf-8')), header=header_rows)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
reports_df[report_name]['Media Cost'].sum()

25242.300000000003

In [17]:
for report_name, report_df in reports_df.items():
    report_df.drop(report_df.index[-1], inplace=True)
    report_df.to_csv(
        '/home/jubauser1/jzou/dcm_account7252/winery_weekly/dcm_reports/%s_20170602_to_%s.csv'%(report_name, end_date), index=False)

In [15]:
reports_df['dma']

Report Fields
Designated Market Area (DMA) Placement                                          Activity  Activity ID Impressions Clicks DBM Cost (Account Currency) Media Cost  Total Conversions  Total Revenue
(not set)                    ffc winery_q2-q3 diamond digital_programmatic_m... (not set) (not set)   0           0      34083.52                    0.000000000 0.00                 0.000000000
                             ffc winery_q2-q3 diamond digital_programmatic_m... (not set) (not set)   0           0      15683.99                    0.000000000 0.00                 0.000000000
...                                                                                                                                                                                           ...
Zanesville                   ffc winery_q2-q3 diamond digital_programmatic_m... (not set) (not set)   80          0      0.00                        0.000000000 0.00                 0.000000000
                             ffc winery_q2-q3 diamond digital_programmatic_m... (not set) (not set)   146         0      0.00                        0.000000000 0.00                 0.000000000
                             ffc winery_q2-q3 diamond digital_programmatic_m... (not set) (not set)   44          0      0.00                        0.000000000 0.00                 0.000000000

[42288 rows x 1 columns]

In [46]:
'40009746568' in set(df['DBM Site ID'])

True

In [24]:
df[df['placement_cat'] == 'geo-fence'].groupby('Designated Market Area (DMA)')

,Designated Market Area (DMA),Placement,Activity,Activity ID,Impressions,Clicks,DBM Cost (Account Currency),Media Cost,Total Conversions,Total Revenue,placement_cat
0,(not set),ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),0,0,14060.08,0.0,0.0,0.0,geo-fence
1,(not set),ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),0,0,7165.82,0.0,0.0,0.0,geo-fence
132,Abilene/Sweetwater,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),3222,3,0.00,0.0,0.0,0.0,geo-fence
...,...,...,...,...,...,...,...,...,...,...,...
38880,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),1210,1,0.00,0.0,0.0,0.0,geo-fence
38881,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),549,4,0.00,0.0,0.0,0.0,geo-fence
38882,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,Store Locator,6165557,0,0,0.00,0.0,1.0,0.0,geo-fence


In [25]:
set(df['Activity'])

{'(not set)',
 'All Products',
 'Checkout',
 'Diamond - Cabernet Sauvignon page',
 'Diamond - Chardonnay page',
 'Diamond - Claret page',
 'Diamond - Diamond Red Blend page',
 'Diamond - Malbec page',
 'Diamond - Merlot page',
 'Diamond - Pavilion page',
 'Diamond - Pinot Grigio page',
 'Diamond - Pinot Noir page',
 'Diamond - Sauvignon Blanc page',
 'Diamond - Syrah Shiraz page',
 'Diamond - Zinfandel page',
 'Diamond Adventure landing page',
 'Diamond Quiz',
 'FFC Home Page',
 'Membership',
 'Membership-Join now',
 'Our Wines Diamond Collection',
 'Shop - Diamond Collection Wines',
 'Shop Now',
 'Shop online - Wine',
 'Shopping Cart',
 'Store Locator',
 'Thank You Page',
 'Visit Location'}

In [21]:
def get_placement_cat(p):
    if 'direct' in p and 'prospecting' not in p and 'retargeting' not in p and 'geo-fence' not in p:
        return 'direct'
    elif 'direct' not in p and 'prospecting' in p and 'retargeting' not in p and 'geo-fence' not in p:
        return 'prospecting'
    elif 'direct' not in p and 'prospecting' not in p and 'retargeting' in p and 'geo-fence' not in p:
        return 'retargeting'
    elif 'direct' not in p and 'prospecting' not in p and 'retargeting' not in p and 'geo-fence' in p:
        return 'geo-fence'
    else:
        return 'others'
    
for report_name, report_df in reports_df.items():
    # drop Grand Total row
    report_df.drop(report_df.index[-1], inplace=True)
    report_df['placement_cat'] = report_df['Placement'].apply(get_placement_cat)

In [29]:
for placement_cat, placement_group in df.groupby('placement_cat'):
    print(placement_cat)

direct
geo-fence
prospecting
retargeting


In [27]:
count_by_view_act[placement_cat] = dict()
c = placement_group[placement_group['Activity'] != 'Thank You Page']
t = placement_group[placement_group['Activity'] == 'Thank You Page']
for view in views_act:
    count_by_view_act[placement_cat][view] = dict()
    # conversion
    conversion = c.groupby(view)['Total Conversions'].sum()
    count_by_view_act[placement_cat][view]['HVA'] = dict(conversion)
    # transaction
    transaction = t.groupby(view)['Total Conversions'].sum()
    count_by_view_act[placement_cat][view]['Transaction'] = dict(transaction) 
    for hva in hvas:
        h = placement_group[placement_group['Activity'] == hva]
        hva_count = h.groupby(view)['Total Conversions'].sum()
        count_by_view_act[placement_cat][view]['hva_%s'%hva] = dict(hva_count) 

,Designated Market Area (DMA),Placement,Activity,Activity ID,Impressions,Clicks,DBM Cost (Account Currency),Media Cost,Total Conversions,Total Revenue,placement_cat
42,(not set),ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),0,0,164.39,0.0,0.0,0.0,retargeting
43,(not set),ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),0,0,376.19,0.0,0.0,0.0,retargeting
44,(not set),ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),0,0,1025.43,0.0,0.0,0.0,retargeting
...,...,...,...,...,...,...,...,...,...,...,...
38928,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),78,0,0.00,0.0,0.0,0.0,retargeting
38929,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),143,0,0.00,0.0,0.0,0.0,retargeting
38930,Zanesville,ffc winery_q2-q3 diamond digital_programmatic_...,(not set),(not set),43,0,0.00,0.0,0.0,0.0,retargeting


In [31]:
c = placement_group[placement_group['Activity'] != 'Thank You Page']
t = placement_group[placement_group['Activity'] == 'Thank You Page']

In [32]:
count_by_view_act = dict()
count_by_view_act[placement_cat] = dict()

In [33]:
hvas = [
    'All Products',
    'Checkout',
    'Diamond - Cabernet Sauvignon page',
    'Diamond - Chardonnay page',
    'Diamond - Claret page',
    'Diamond - Diamond Red Blend page',
    'Diamond - Malbec page',
    'Diamond - Merlot page',
    'Diamond - Pavilion page',
    'Diamond - Pinot Grigio page',
    'Diamond - Pinot Noir page',
    'Diamond - Sauvignon Blanc page',
    'Diamond - Syrah Shiraz page',
    'Diamond - Zinfandel page',
    'Diamond Adventure landing page',
    'Diamond Quiz',
    'FFC Home Page',
    'Membership',
    'Membership-Join now',
    'Our Wines Diamond Collection',
    'Shop - Diamond Collection Wines',
    'Shop Now',
    'Shop online - Wine',
    'Shopping Cart',
    'Store Locator',
    'Visit Location']

In [34]:
view = 'Designated Market Area (DMA)'
count_by_view_act[placement_cat][view] = dict()
# conversion
conversion = c.groupby(view)['Total Conversions'].sum()
count_by_view_act[placement_cat][view]['HVA'] = dict(conversion)
# transaction
transaction = t.groupby(view)['Total Conversions'].sum()
count_by_view_act[placement_cat][view]['Transaction'] = dict(transaction) 
for hva in hvas:
    h = placement_group[placement_group['Activity'] == hva]
    hva_count = h.groupby(view)['Total Conversions'].sum()
    count_by_view_act[placement_cat][view]['hva_%s'%hva] = dict(hva_count) 

In [37]:
pd.DataFrame(count_by_view_act['retargeting']['Designated Market Area (DMA)'])

,HVA,Transaction,hva_All Products,hva_Checkout,hva_Diamond - Cabernet Sauvignon page,hva_Diamond - Chardonnay page,hva_Diamond - Claret page,hva_Diamond - Diamond Red Blend page,hva_Diamond - Malbec page,hva_Diamond - Merlot page,...,hva_FFC Home Page,hva_Membership,hva_Membership-Join now,hva_Our Wines Diamond Collection,hva_Shop - Diamond Collection Wines,hva_Shop Now,hva_Shop online - Wine,hva_Shopping Cart,hva_Store Locator,hva_Visit Location
(not set),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abilene/Sweetwater,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albany-Ga,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Youngstown,4.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Yuma/El Centro,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zanesville,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# create report

In [9]:
common_filter = [{'dimensionName': 'dfa:advertiser', 'id': '6171870'}, 
                 {'dimensionName': 'dfa:campaign', 'id': '11222752'},
                 {'dimensionName': 'dfa:campaign', 'id': '11233131'},
                 {'dimensionName': 'dfa:campaign', 'id': '11243310'},
                 {'dimensionName': 'dfa:campaign', 'id': '20073454'},
                 {'dimensionName': 'dfa:campaign', 'id': '20093796'},
                 ]
common_date_range = {'startDate': '2017-07-02', 'endDate': '2017-07-04'}
common_metric = ['dfa:costPerClick']

report_cpm = {
    'name': 'cpm',
    'type': 'STANDARD',
    'criteria': { 
        'dimensionFilters': common_filter,
        'dateRange': common_date_range,
        'dimensions': [{'name': 'dfa:date'},
                       #{'name': 'dfa:paidSearchKeyword'}
                      ],
        'metricNames': common_metric
    }
}

# Construct the request.
request = service.reports().insert(profileId=profile_id, body=report_cpm)

# Execute request and print response.
response = request.execute()

print ('Created %s report with ID %s and name "%s".'
       % (response['type'], response['id'], response['name']))

Created STANDARD report with ID 102491903 and name "cpm".


# get report info

In [ ]:
'state_city': '97262853',
    'dma': '97263173',
    #'cpm': '101006950',
    #'clickThroughUrl': '97264726',
    'dbmLineItem': '97349012',
    'dbmSite': '97466324',

In [5]:
report_id = 97466324
request = service.reports().get(profileId=profile_id, reportId=report_id)
report = request.execute()
report

{'accountId': '7252',
 'criteria': {'dateRange': {'endDate': '2017-07-25',
   'kind': 'dfareporting#dateRange',
   'startDate': '2017-06-02'},
  'dimensionFilters': [{'dimensionName': 'dfa:advertiser',
    'etag': '"uHT-XyJURmtBI_Tgzr-e7kL7Lcw/93Fx-M6Wk480jKcdIFuuiTkUCO4"',
    'id': '6171870',
    'kind': 'dfareporting#dimensionValue',
    'matchType': 'EXACT'},
   {'dimensionName': 'dfa:campaign',
    'etag': '"uHT-XyJURmtBI_Tgzr-e7kL7Lcw/kYRsFNswLNGjKgwVZxUHADR39eM"',
    'id': '11527750',
    'kind': 'dfareporting#dimensionValue',
    'matchType': 'EXACT'},
   {'dimensionName': 'dfa:country',
    'etag': '"uHT-XyJURmtBI_Tgzr-e7kL7Lcw/Cz0bfVKsYyaOtQWzRuGf3Vkwt0Q"',
    'id': '256',
    'kind': 'dfareporting#dimensionValue',
    'matchType': 'EXACT'}],
  'dimensions': [{'kind': 'dfareporting#sortedDimension',
    'name': 'dfa:dbmSite'},
   {'kind': 'dfareporting#sortedDimension', 'name': 'dfa:dbmSiteId'},
   {'kind': 'dfareporting#sortedDimension', 'name': 'dfa:placement'},
   {'kind

# update report

In [94]:
report['criteria']['dimensions']

[{'kind': 'dfareporting#sortedDimension', 'name': 'dfa:date'}]

In [95]:
report['criteria']['metricNames']

['dfa:effectiveCpm']

In [96]:
# get report info
request_get_report = service.reports().get(profileId=profile_id, reportId=report_id)
report = request_get_report.execute()

# update end_date

report['criteria']['dateRange']['startDate'] = '2017-06-02'
report['criteria']['dateRange']['endDate'] = '2017-06-04'
report['criteria']['dimensionFilters'] = [
    #{'dimensionName': 'dfa:advertiser', 'id': '6171870'},
    {'dimensionName': 'dfa:campaign', 'id': '11222752'}, # quancast, critire
    #{'dimensionName': 'dfa:campaign', 'id': '11233131'}, # fb
    #{'dimensionName': 'dfa:campaign', 'id': '11243310'},
    {'dimensionName': 'dfa:campaign', 'id': '20073454'}, # quancast, critire
    #{'dimensionName': 'dfa:campaign', 'id': '20093796'} # fb
]
report['criteria']['dimensions'] = [
    {'name': 'dfa:date'},
    #{'name': 'dfa:keyword'},
    #{'name': 'dfa:paidSearchEngineAccount'},
    #{'name': 'dfa:paidSearchAd'},
    #{'name': 'dfa:paidSearchAdvertiser'},
    #{'name': 'dfa:paidSearchAgency'},
    #{'name': 'dfa:paidSearchCampaign'},
    #{'name': 'dfa:paidSearchAdGroup'},
    #{'name': 'dfa:paidSearchMatchType'},
    #{'name': 'dfa:paidSearchKeyword'}
]
report['criteria']['metricNames'] = [
    'dfa:costPerClick',
    'dfa:mediaCost',
    'dfa:clicks',
    'dfa:dbmCost',
    'dfa:impressions'
]
request_update_report = service.reports().update(profileId=profile_id, reportId=report_id, body=report)
response = request_update_report.execute()